In [1]:
import pandas as pd


In [2]:
woreda_mapped_path = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv'
woreda_mapped_df = pd.read_csv(woreda_mapped_path)

In [3]:
woreda_mapped_df = woreda_mapped_df.set_index(['RegionName', 'ZoneName', 'WoredaName'])

In [4]:
woreda_mapped_df.head()

WoredaLat  WoredaLon  \
RegionName  ZoneName    WoredaName                           
Addis Ababa Addis Ababa Addis Ketema   9.034698  38.726558   
                        Akaki Kality   8.897286  38.803770   
                        Arada          9.035681  38.754684   
                        Bole           8.980035  38.839082   
                        Gulele         9.071479  38.744860   

                                                        match_0  \
RegionName  ZoneName    WoredaName                                
Addis Ababa Addis Ababa Addis Ketema  addis ketema kifle ketema   
                        Akaki Kality                      akaki   
                        Arada                arada kifle ketema   
                        Bole                  bole kifle ketema   
                        Gulele              gulele kifle ketema   

                                                                            match_1  \
RegionName  ZoneName    WoredaName                                                    
Addis Ababa Addis Ababa Addis Ketema  addis ketema operational woreda health office   
                        Akaki Kality                                  akaki - kalit   
                        Arada                                          arada woreda   
                        Bole                                                  bolle   
                        Gulele                                              gulelie   

                                                    match_2  \
RegionName  ZoneName    WoredaName                            
Addis Ababa Addis Ababa Addis Ketema  Addis Ketema Sub City   
                        Akaki Kality           akaki kaliti   
                        Arada                Arada Sub City   
                        Bole                  Bole Sub City   
                        Gulele                      gulelle   

                                                        match_3  \
RegionName  ZoneName    WoredaName                                
Addis Ababa Addis Ababa Addis Ketema                        NaN   
                        Akaki Kality  akaki kality kifle ketema   
                        Arada                               NaN   
                        Bole                                NaN   
                        Gulele                          gullele   

                                                    match_4 match_5 match_6  \
RegionName  ZoneName    WoredaName                                            
Addis Ababa Addis Ababa Addis Ketema                    NaN     NaN     NaN   
                        Akaki Kality  Akaki Kaliti Sub City     NaN     NaN   
                        Arada                           NaN     NaN     NaN   
                        Bole                            NaN     NaN     NaN   
                        Gulele              Gulele Sub City     NaN     NaN   

                                     match_7 match_8 match_9 match_10  \
RegionName  ZoneName    WoredaName                                      
Addis Ababa Addis Ababa Addis Ketema     NaN     NaN     NaN      NaN   
                        Akaki Kality     NaN     NaN     NaN      NaN   
                        Arada            NaN     NaN     NaN      NaN   
                        Bole             NaN     NaN     NaN      NaN   
                        Gulele           NaN     NaN     NaN      NaN   

                                     match_11 match_12 match_13 match_14  \
RegionName  ZoneName    WoredaName                                         
Addis Ababa Addis Ababa Addis Ketema      NaN      NaN      NaN      NaN   
                        Akaki Kality      NaN      NaN      NaN      NaN   
                        Arada             NaN      NaN      NaN      NaN   
                        Bole              NaN      NaN      NaN      NaN   
                        Gulele            NaN      NaN      NaN      NaN   

            

In [5]:
lat, lon = woreda_mapped_df.ix['Addis Ababa', 'Addis Ababa', 'Addis Ketema'][['WoredaLat', 'WoredaLon']]
lon

<h1> Function to assign Lat Lons at any level in Ethiopia data </h1>

In [5]:
# Generate the geo_key given the available column names for woreda, zone, region.

region_mapped_dir = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data'

mapped_filenames = ['woreda_mapped.csv', 'zone_mapped.csv', 'region_mapped.csv']

#COLUMNS = ['RegionName', 'ZoneName', 'WoredaName']
#level_map = {1: 'RegionKey', 2: 'ZoneKey', 3: 'WoredaKey'}


def make_geokey(rzw_list):
    ''' Input a list of region, zone, woreda name.'''
    if len(rzw_list) > 0:
        end_dash = '__' * (4 - len(rzw_list))
        return '%s%s' % ('__'.join(rzw_list).lower(), end_dash)
    else:
        return ''

def process_geokeys(df_mapped):
    name_cols = [column for column in df_mapped.columns if 'Name' in column]
    df_mapped['GeoKey'] = \
        df_mapped[name_cols].apply(lambda x: make_geokey(x), axis=1)
        
    print name_cols
    for level_num, column in enumerate(name_cols):
        print level_num, column
        levels = name_cols[:level_num + 1]
        levelkey = '%sKey' % column.split('Name')[0]
        print levelkey
        df_mapped[levelkey] = \
            df_mapped[levels].apply(lambda x: make_geokey(x), axis=1)
    
    return df_mapped

 
def selfjoin_fill_latlons(dataframe):
    POSTFIX_LIST = ['Key', 'Lat', 'Lon']
    for level in ['Region', 'Zone']:
        level_key = '%sKey' % level
        column_list = ['%s%s' % (level, postfix) for postfix in POSTFIX_LIST]
        dataframe = pd.merge(dataframe[column_list].dropna(), dataframe, how='right', on=level_key, suffixes={'', '_y'})
    # Clean up.
    columns_to_drop = [column for column in dataframe.columns if '_y' in column]
    dataframe.drop(columns_to_drop, axis=1, inplace=True)
    return dataframe
    
    
# Initialize empty dataframe. Then loop through all mapped.csv files and append to it.
latlon_maps = pd.DataFrame()


for filename in mapped_filenames:
    df_mapped = pd.read_csv('%s/%s' % (region_mapped_dir, filename), dtype=str)
    match_columns = [column for column in df_mapped.columns if 'match' in column.lower()]
    df_mapped.drop(match_columns, axis=1, inplace=True)
    df_mapped = process_geokeys(df_mapped)
    latlon_maps = latlon_maps.append(df_mapped, ignore_index=True)

latlon_maps = selfjoin_fill_latlons(latlon_maps)
latlon_maps.set_index('GeoKey', inplace=True)

['RegionName', 'ZoneName', 'WoredaName']
0 RegionName
RegionKey
1 ZoneName
ZoneKey
2 WoredaName
WoredaKey
['RegionName', 'ZoneName']
0 RegionName
RegionKey
1 ZoneName
ZoneKey
['RegionName']
0 RegionName
RegionKey


In [8]:
latlon_maps.head()

,ZoneKey,ZoneLat,ZoneLon,RegionKey,RegionLat,RegionLon,RegionName,WoredaKey,WoredaLat,WoredaLon,WoredaName,ZoneName
GeoKey,,,,,,,,,,,,
addis ababa__addis ababa__addis ketema__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__addis ketema__,9.034698212854872,38.726558114195036,Addis Ketema,Addis Ababa
addis ababa__addis ababa__akaki kality__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__akaki kality__,8.89728556566618,38.803769890705,Akaki Kality,Addis Ababa
addis ababa__addis ababa__arada__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__arada__,9.035681311338866,38.7546835191774,Arada,Addis Ababa
addis ababa__addis ababa__bole__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__bole__,8.98003533307717,38.83908245092445,Bole,Addis Ababa
addis ababa__addis ababa__gulele__,addis ababa__addis ababa____,8.980483001254077,38.78553834507309,addis ababa______,8.980483001254077,38.78553834507309,Addis Ababa,addis ababa__addis ababa__gulele__,9.071479416494498,38.744859937113176,Gulele,Addis Ababa


<h1> Save </h1>

In [12]:
latlon_maps.to_csv('geokey_latlon.csv')

<h1> Usage. Lookup geokey and return a dictionary with with lat lon info. </h1>

In [6]:
# Input a geokey. Output a dictionary wit lat lons will be returned as a string.

latlons = ['RegionLat', 'RegionLon', 'ZoneLat', 'ZoneLon', 'WoredaLat', 'WoredaLon']

geokey = 'tigray______'
latlon_dict = {}

if geokey in latlon_maps.index:
    latlon_dict = latlon_maps.loc[geokey][latlons].dropna().to_dict()
latlon_dict
    

{'RegionLat': '13.777114742142253', 'RegionLon': '38.43867332133363'}